## 로지스틱 회귀(분류)-연습문제

### 로지스틱 회귀(분류) 알고리즘 사용 모델링 예제
- Body Peformance 데이터 사용 

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

body = pd.read_csv('../../data/bodyPerformance.csv')
body.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [2]:
# class가 A, B, C, D로 되어 있는데
body['class'].value_counts()

class
C    3349
D    3349
A    3348
B    3347
Name: count, dtype: int64

In [3]:
# A인 경우 1, A가 아닌 경우 0으로 변경 (이진 분류)
body['class'] = body['class'].apply(lambda x: 1 if x == 'A' else 0)

In [4]:
body['class'].value_counts()

class
0    10045
1     3348
Name: count, dtype: int64

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# 피처 데이터
feature_data = body.drop('class', axis=1)
feature_data.head(2)

# 레이블 데이터
labels = body['class']
labels.head(5)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0


0    0
1    1
2    0
3    0
4    0
Name: class, dtype: int64

In [10]:
# 'Gender' 열을 이진 값으로 변환, 남자는 0, 여자는 1
feature_data['gender'] = feature_data['gender'].map({'M': 0, 'F': 1})

#### 스케일링 / 데이터 세트 분리

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 피처  표준 스케일링
scaler = StandardScaler()
data_scaled = scaler.fit_transform(feature_data)

# 데이터 세트 분리
X_train, X_test, y_train, y_test = train_test_split(data_scaled,
                                                    labels,
                                                    stratify=labels, 
                                                    random_state=11)

#### 학습 / 예측 / 평가

In [12]:
# 로지스틱 회귀를 이용하여 학습/예측/평가

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression(solver='liblinear')
# solver : 최적화 문제에 사용하는 알고리즘
# liblinear : 적은 양의 데이터에 적합

lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)

# 정확도
print('정확도 :', accuracy_score(y_test, y_pred))

LogisticRegression(solver='liblinear')

정확도 : 0.8515974917885936


#### 하이퍼 파라미터 튜닝

In [16]:
# 하이퍼 파라미터 설정 : 최적 평균 정확도 확인
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_iter':[50, 100, 300, 500, 1000],
    'penalty' : ['l2', 'l1'],
    'C' : [0.01, 0.05, 0.1, 1, 5, 10, 20]
}

# GridSearchCV 이용해서 최적의 하이퍼 파라미터 찾음
# 교차 검증 폴드 수 : cv=3, cv=5 .. 바꿔가면서 결과 비교
grid_rlclf = GridSearchCV(lr_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_rlclf.fit(X_train, y_train)


print('GridSearchCV 최적 하이퍼 파라미터 : ', grid_rlclf.best_params_)
print('GridSearchCV 최고 정확도 : {0:.4f} ', format(grid_rlclf.best_score_))

# 최고 점수를 낸 파라미터를 가진 모형(모델)
best_rllf = grid_rlclf.best_estimator_

# 최적 하이퍼  파라미터로 학습된 Estimator 예측/평가 수행
y_pred = best_rllf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('하이퍼 파라미터 튜닝을 끝낸 최종 정확도 : {0:.4f} ', format(accuracy))

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [0.01, 0.05, 0.1, 1, 5, 10, 20],
                         'max_iter': [50, 100, 300, 500, 1000],
                         'penalty': ['l2', 'l1']},
             scoring='accuracy')

GridSearchCV 최적 하이퍼 파라미터 :  {'C': 10, 'max_iter': 50, 'penalty': 'l2'}
GridSearchCV 최고 정확도 : {0:.4f}  0.8457776162646576
하이퍼 파라미터 튜닝을 끝낸 최종 정확도 : {0:.4f}  0.8515974917885936


In [19]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score 

def get_clf_eval(y_test, y_pred, pred_proba) :
    cm = confusion_matrix(y_test, y_pred) # 혼동행렬(오차행렬)
    accuracy = accuracy_score(y_test, y_pred) # 정확도
    precision = precision_score(y_test, y_pred) # 정밀도
    recall = recall_score(y_test, y_pred)# 재현율
    f1 = f1_score(y_test, y_pred) # F1 스코어
    roc_auc = roc_auc_score(y_test, pred_proba) # ROC_AUC 스코어
    
    # 출력 
    # 오차행렬
    print("오차행렬 : ", cm)
    # 정확도, 정밀도, 재현율, F1
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision:.4f},  재현율: {recall:.4f}, F1 Score: {f1:.4f}, ROC_AUC : {roc_auc:.4f}")

In [21]:
# 모델 생성
# 하이퍼 파라미터 튜닝에서 찾은 최적의 파라미터로 모델 생성/학습/예측/평가
lr_clf = LogisticRegression(solver='liblinear', C=10, penalty='l2', max_iter= 50)
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)

pred_proba = lr_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, y_pred, pred_proba)

LogisticRegression(C=10, max_iter=50, solver='liblinear')

오차행렬 :  [[2322  190]
 [ 307  530]]
정확도: 0.8516, 정밀도: 0.7361,  재현율: 0.6332, F1 Score: 0.6808, ROC_AUC : 0.9238


In [ ]:
############################################################

## 다중 분류

In [34]:
body = pd.read_csv('../../data/bodyPerformance.csv')
body.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [35]:
# class A, B, C, D 전부 다 사용
body['class'].value_counts()

class
C    3349
D    3349
A    3348
B    3347
Name: count, dtype: int64

In [36]:
# class : A, B, C, D 을 0, 1, 2, 3으로 변경하여 사용
body['class'] = body['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3})
body['class'].value_counts()

class
2    3349
3    3349
0    3348
1    3347
Name: count, dtype: int64

In [37]:
# 피처 데이터
feature_data = body.drop('class', axis=1)
feature_data.head(2)

# 레이블 데이터
labels = body['class']
labels.head(5)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0


0    2
1    0
2    2
3    1
4    1
Name: class, dtype: int64

In [38]:
# 'Gender' 열을 이진 값으로 변환, 남자는 0, 여자는 1
feature_data['gender'] = feature_data['gender'].map({'M': 0, 'F': 1})

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 피처  표준 스케일링
scaler = StandardScaler()
data_scaled = scaler.fit_transform(feature_data)

# 데이터 세트 분리
X_train, X_test, y_train, y_test = train_test_split(data_scaled,
                                                    labels,
                                                    stratify=labels, 
                                                    random_state=11)

In [40]:
lr_clf = LogisticRegression(solver='lbfgs', max_iter= 50, multi_class='multinomial', C=10, penalty='l2')
# solver : 최적화 문제에 사용하는 알고리즘

lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)

# 정확도
print('정확도 :', accuracy_score(y_test, y_pred))

LogisticRegression(C=10, max_iter=50, multi_class='multinomial')

정확도 : 0.6246640788295014


In [41]:
precision_score(y_test, y_pred, average='micro') # 정밀도
precision_score(y_test, y_pred, average='macro')
precision_score(y_test, y_pred, average='weighted')

0.6246640788295014

0.6212877919853144

0.6213367364678395

In [42]:
recall_score(y_test, y_pred, average='micro') # 재현율
recall_score(y_test, y_pred, average='macro')
recall_score(y_test, y_pred, average='weighted')

0.6246640788295014

0.6246161281768334

0.6246640788295014

In [43]:
f1_score(y_test, y_pred, average='micro') # F1 스코어
f1_score(y_test, y_pred, average='macro')
f1_score(y_test, y_pred, average='weighted')

0.6246640788295014

0.6227091415571886

0.6227576616295698